In [ ]:
import os
import sys
import yaml

sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))
sys.path.append(os.path.abspath(os.path.join('..', 'model/autoencoder/')))

# Define the path to the config.yaml file
config_path = os.path.abspath(os.path.join('..', 'configs', 'config.yaml'))

# Load the YAML file
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)



In [ ]:
from capture_data import NetworkTrafficCollector

raw_path = config['data']['raw_path']
print(raw_path)
collection = NetworkTrafficCollector(collection_point="Wi-Fi 2", file_path=raw_path)
#collection.start_capture(timeout=60)

In [ ]:
from capture_data import NetworkTrafficCollector

new_raw_path = config['data']['new_raw_path']
print(new_raw_path)
new_collection = NetworkTrafficCollector(collection_point="Wi-Fi 2", file_path=new_raw_path)
#new_collection.start_capture(timeout=60)

In [ ]:
from preprocess_data import process_packets

raw_path = config['data']['raw_path']
processed_path = config['data']['processed_path']
#process_packets(raw_path, processed_path)

new_raw_path = config['data']['new_raw_path']
new_processed_path = config['data']['new_processed_path']
#process_packets(new_raw_path, new_processed_path)

In [ ]:
from data_loader import load_data_from_h5, create_data_loaders, create_batches

batch_size = config['training']['batch_size']
data_tensor = load_data_from_h5(processed_path)
batch_dataset = create_batches(data_tensor, batch_size)
data_loader = create_data_loaders(batch_dataset)

In [ ]:
from train_model import train_autoencoder

train_autoencoder(data_loader, input_dim=6, epochs=1)

In [ ]:
from train_model import test_autoencoder
from tensorflow.keras.losses import MeanSquaredError

test_autoencoder(data_loader)

In [ ]:
import numpy as np

def generate_fake_data(num_samples=100, anomaly=False):
    normal_data = np.array([
        [5.7344942e-06, 9.6054333e-01, 3.2170895e-01, 7.3307459e-03, 0.0000000e+00, 4.2857143e-01],
        [5.7344942e-06, 9.6054333e-01, 3.2170895e-01, 7.3307459e-03, 0.0000000e+00, 0.0000000e+00],
        [5.7344942e-06, 9.6054333e-01, 3.2170895e-01, 7.3307459e-03, 0.0000000e+00, 0.0000000e+00],
        [5.7344942e-06, 9.6054333e-01, 3.2170895e-01, 7.3307459e-03, 0.0000000e+00, 0.0000000e+00]
    ], dtype=np.float32)
    
    # If anomaly, introduce random noise
    if anomaly:
        noise = np.random.normal(0, 1, normal_data.shape)
        data = normal_data + noise
    else:
        data = normal_data
    
    data = np.repeat(data[np.newaxis, ...], num_samples, axis=0)
    data = data.reshape(num_samples * 4, 6)  # Reshape to (num_samples * 4, 6)
    return data

In [ ]:

normal_data = generate_fake_data(num_samples=100, anomaly=False)
anomalous_data = generate_fake_data(num_samples=100, anomaly=True)

In [ ]:
import tensorflow as tf

def load_autoencoder_model(model_path):
    return tf.keras.models.load_model(model_path, custom_objects={'MeanSquaredError': tf.keras.losses.MeanSquaredError})

# Example usage:
model_path = 'C:/Users/theob/Code/Refactored MLSec/notebooks/models/autoencoder/trained_models/autoencoder_model.keras'
autoencoder = load_autoencoder_model(model_path)

In [12]:
from detect_anomalies import make_predictions

# Predict on normal data
normal_mse = make_predictions(autoencoder, normal_data)
print(f"Normal Data MSE: {sum(normal_mse)}")

# Predict on anomalous data
anomalous_mse = make_predictions(autoencoder, anomalous_data)
print(f"Anomalous Data MSE: {sum(anomalous_mse)}")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step
Normal Data MSE: 3.973746232688427
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step
Anomalous Data MSE: 172.14430529922652
